In [6]:
!pip install pymysql

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import pymysql
from datetime import datetime
import requests

In [8]:
PWA_DB_HOST_V12CHAT_READ="preproduction-db-1-instance-1.c7nkisio0atn.us-west-2.rds.amazonaws.com"
PWA_DB_PORT_V12CHAT_READ="3306"
PWA_DB_DATABASE_V12CHAT_READ="v12_conversation"
PWA_DB_USERNAME_V12CHAT_READ="v12_conversation"
PWA_DB_PASSWORD_V12CHAT_READ="V12_c0nvers&tion_2023"
#############################################
PWA_DB_HOST_V12CHAT_WRITE="preproduction-db-1-instance-1.c7nkisio0atn.us-west-2.rds.amazonaws.com"
PWA_DB_PORT_V12CHAT_WRITE="3306"
PWA_DB_DATABASE_V12CHAT_WRITE="v12_conversation"
PWA_DB_USERNAME_V12CHAT_WRITE="v12_conversation"
PWA_DB_PASSWORD_V12CHAT_WRITE="V12_c0nvers&tion_2023"
#############################################
PWA_DB_HOST_V12_ACCOUNT_READ="preproduction-db-1-instance-1.c7nkisio0atn.us-west-2.rds.amazonaws.com"
PWA_DB_PORT_V12_ACCOUNT_READ="3306"
PWA_DB_DATABASE_V12_ACCOUNT_READ="v12_account"
PWA_DB_USERNAME_V12_ACCOUNT_READ="v12_account"
PWA_DB_PASSWORD_V12_ACCOUNT_READ="PHR6aHLcHx!aM6Wk"

In [9]:
import jwt
import datetime as dt
import jwt

import os
from dotenv import load_dotenv
load_dotenv()
secret_key = str(os.getenv("secret_key", ""))

def create_token():
    module_name = "bot"
    
    payload = {
        "iss": module_name,  # Issuer
        "iat": dt.datetime.utcnow(),  # Issued At
        "exp": dt.datetime.utcnow() + dt.timedelta(minutes=2),  # Expiration (30 min)
        "nbf": dt.datetime.utcnow(),  # Not Before
        "jti": module_name,  # JWT ID
        "sub": module_name  # Subject
    }

    # Encode the JWT token
    token = jwt.encode(payload, secret_key, algorithm="HS256")

    # Set up headers with the JWT token
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    
    return headers


def create_token():
    module_name='bot'
    payload = {
        "iss": module_name,  # Issuer
        "iat": dt.datetime.utcnow(),  # Issued At
        "exp": dt.datetime.utcnow() + dt.timedelta(minutes=2),  # Expiration
        "nbf": dt.datetime.utcnow(),  # Not Before
        "jti": module_name,  # JWT ID
        "sub": module_name  # Subject
    }

    # Encode the JWT
    token = jwt.encode(payload, secret_key, algorithm="HS256")

    # Set up headers with the JWT token
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    return headers


In [10]:
import pymysql
from datetime import datetime

class DataBase:
    # Connection to any database using HOST, USER, PASSWORD, NAME
    def connexion(self, host, user, password, database):
        try:
            conn = pymysql.connect(
                host=host,
                user=user,
                password=password,
                database=database
            )
            return conn
        except pymysql.Error as e:
            error_mes = 'Connexion function => ' + str(e)
            now = datetime.now()
            text = f'{{"@timestamp":"{now}","log.level":"ERROR","app_name":"Bot","Channel":"PWA","message":"{error_mes}"}}'
            print(text)  # Consider logging it to a file instead
            return None

    # Get any value from database using SELECT query
    def readQuery(self, conn, query, data=None):
        try:
            with conn.cursor() as cur:
                if data is not None:
                    cur.execute(query, data)
                else:
                    cur.execute(query)
                rows = cur.fetchall()
            return rows
        except pymysql.Error as e:
            error_mes = 'readQuery function => ' + str(e)
            now = datetime.now()
            text = f'{{"@timestamp":"{now}","log.level":"ERROR","app_name":"Bot","Channel":"PWA","message":"{error_mes}"}}'
            print(text)
            return None

    # Insert any value in database using INSERT query
    def writeQuery(self, conn, query, data):
        try:
            with conn.cursor() as cursor:
                resp = cursor.execute(query, data)
                conn.commit()
            return resp
        except pymysql.Error as e:
            error_mes = 'writeQuery function => ' + str(e)
            now = datetime.now()
            text = f'{{"@timestamp":"{now}","log.level":"ERROR","app_name":"Bot","Channel":"PWA","message":"{error_mes}"}}'
            print(text)
            return 0

    # Update any table in database with UPDATE query
    def updateQuery(self, conn, query, data):
        return self.writeQuery(conn, query, data)

    # Delete rows from table
    def deleteQuery(self, conn, query, data):
        return self.writeQuery(conn, query, data)

    # Write to tracking conversation
    def writeTracking(self, conn, data):        
        query = """
        INSERT INTO tracking_conversation 
        (intent, step, understand, date_time, proposed_time, list_time, year, make, miles, vehicle_id, vehicle_name, finished, take_by_human, lead_id, dealer_id, date_without, last_response) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        return self.writeQuery(conn, query, data)

    # Write to tracking conversation SMS
    def writeTrackingSmsBdc(self, conn, data):        
        query = """
        INSERT INTO tracking_sms_conversations 
        (intent, step, date_time, proposed_time, list_time, vehicle_id, vehicle_name, lead_id, dealer_id, last_message, last_message_date, end_conversation, date_without) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        return self.writeQuery(conn, query, data)


In [11]:
db = DataBase()
dealer_id=40255

_shipping

In [12]:
def _shipping(DB, dealer_id):
    shipping = None
    shippingQuery = "SELECT is_shipping FROM chat_settings WHERE user_id = %s"
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    if conn_DBC:
        result = DB.readQuery(conn_DBC, shippingQuery, (dealer_id,))
        if result:
            shipping = result[0][0]
        conn_DBC.close()  # Ensure connection is closed
    return shipping

dealership_name

In [13]:
def dealership_name(DB, dealer_id):
    dealer_name = None
    try:
        conn_Prod = DB.connexion(
            PWA_DB_HOST_V12_ACCOUNT_READ,
            PWA_DB_USERNAME_V12_ACCOUNT_READ,
            PWA_DB_PASSWORD_V12_ACCOUNT_READ,
            PWA_DB_DATABASE_V12_ACCOUNT_READ,
        )
        queryPROD = "SELECT company_name FROM dealer_address WHERE user_id = %s"
        res = DB.readQuery(conn_Prod, queryPROD, dealer_id)
        dealer_name = res[0][0]
    except:
        pass
    return dealer_name

dealer_time_zone

In [14]:
def dealer_time_zone(DB, dealer_id):
    query_time = "SELECT time_zone FROM `dealer_address` WHERE user_id= %s"
    conn_Prod = DB.connexion(
        PWA_DB_HOST_V12_ACCOUNT_READ,
        PWA_DB_USERNAME_V12_ACCOUNT_READ,
        PWA_DB_PASSWORD_V12_ACCOUNT_READ,
        PWA_DB_DATABASE_V12_ACCOUNT_READ,
    )
    res_type = DB.readQuery(conn_Prod, query_time, dealer_id)
    timeZone = res_type[0][0]
    return timeZone

address and link of location for dealer

In [15]:
def address_location(DB,dealer_id):
    conn_Prod = DB.connexion(
        PWA_DB_HOST_V12_ACCOUNT_READ,
        PWA_DB_USERNAME_V12_ACCOUNT_READ,
        PWA_DB_PASSWORD_V12_ACCOUNT_READ,
        PWA_DB_DATABASE_V12_ACCOUNT_READ,
    )
    queryPROD = (
        "SELECT  address, city, state_id, zip FROM dealer_address WHERE user_id = %s"
    )
    res = DB.readQuery(conn_Prod, queryPROD, dealer_id)
    address = res[0][0]
    city = res[0][1]
    state_id = res[0][2]
    zipcode = res[0][3]
    querystate = "SELECT  state_name FROM states WHERE id = %s"
    state_res = DB.readQuery(conn_Prod, querystate, state_id)
    state = state_res[0][0]
    addcty = address + ",+" + city
    new_adress = addcty.replace(" ","+")
    dealer_address = address + " " + str(city) + " " + str(state) + " " + str(zipcode)
    link = "https://www.google.com/maps/search/?api=1&query=" + new_adress
    return [dealer_address, link]

get dealer phone

In [16]:
def get_dealer_phone(DB, dealer_id):
    query1 = "SELECT `display_phone_number` FROM `chat_settings` WHERE `user_id`= %s"
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    res_link = DB.readQuery(conn_DBC, query1, dealer_id)
    if res_link[0][0] is not None and res_link[0][0] != "" and res_link[0][0] != "null":
        phone = res_link[0][0]
    else:
        query_phone = "SELECT cell_phone FROM `dealer_address` WHERE user_id=%s"
        conn_Prod = DB.connexion(
            PWA_DB_HOST_V12_ACCOUNT_READ,
            PWA_DB_USERNAME_V12_ACCOUNT_READ,
            PWA_DB_PASSWORD_V12_ACCOUNT_READ,
            PWA_DB_DATABASE_V12_ACCOUNT_READ,
        )
        phone_res = DB.readQuery(conn_Prod, query_phone, dealer_id)
        phone = phone_res[0][0]
    phone = "(" + phone[:3] + ") " + phone[3:6] + "-" + phone[6:]
    return phone

 get inventory url

In [17]:
def inventory_url(DB, dealer_id):
    query1 = "SELECT `is_inventory_internal_link`,`external_inventory_link` FROM `chat_settings` WHERE `user_id`= %s"
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    res_link = DB.readQuery(conn_DBC, query1, dealer_id)
    if res_link[0][0] == 0:
        link = res_link[0][1]
        if link is not None:
            return link
    query2 = "SELECT name FROM domain_name WHERE user_id = %s"
    conn_Prod = DB.connexion(
        PWA_DB_HOST_V12_ACCOUNT_READ,
        PWA_DB_USERNAME_V12_ACCOUNT_READ,
        PWA_DB_PASSWORD_V12_ACCOUNT_READ,
        PWA_DB_DATABASE_V12_ACCOUNT_READ,
    )
    res = DB.readQuery(conn_Prod, query2, dealer_id)
    inventory_link = res[0][0] + "/all-inventory"
    return inventory_link

credit application link 

In [18]:
def financing_url(DB, dealer_id):
    financing_link = None
    financeQuery = "SELECT is_financing, external_financing_link, is_internal_link, type_financing_id, credit_score, other_financing FROM `chat_settings` where user_id=%s"
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    financingRes = DB.readQuery(conn_DBC, financeQuery, dealer_id)
    is_financing = financingRes[0][0]
    external_financing_link = financingRes[0][1]
    is_internal_link = financingRes[0][2]
    finance_type=financingRes[0][3]
    credit_score=financingRes[0][4]
    other_financing=financingRes[0][5]
    if is_financing == 0 or is_financing == '0' :
        return ["This dealer not offer financing", finance_type,credit_score, other_financing ]
    if is_internal_link == 0 or is_internal_link == '0':
        financing_link = external_financing_link
    else:
        query2 = "SELECT name FROM domain_name WHERE user_id = %s"
        conn_Prod = DB.connexion(
            PWA_DB_HOST_V12_ACCOUNT_READ,
            PWA_DB_USERNAME_V12_ACCOUNT_READ,
            PWA_DB_PASSWORD_V12_ACCOUNT_READ,
            PWA_DB_DATABASE_V12_ACCOUNT_READ,
        )
        res = DB.readQuery(conn_Prod, query2, dealer_id)
        financing_link = res[0][0] + "/credit-application"
    return [financing_link, finance_type,credit_score, other_financing ]

welcome message 

In [19]:
def get_welcome_message(DB, dealer_id):
    query = "SELECT is_custom_fbmp_message , content_custom_fbmp_message FROM `chat_settings` WHERE `user_id`=%s"
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    welcome_msg = DB.readQuery(conn_DBC, query, dealer_id)
    if welcome_msg[0][0] == 1 or welcome_msg[0][0] == "1":
        return welcome_msg[0][1]
    else:
        return None

message when informations of car not provide

In [20]:
def message_no_informations(DB, dealer_id):
    query = "SELECT is_custom_no_information , content_custom_no_information	 FROM `chat_settings` WHERE `user_id`=%s"
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    no_infos_msg = DB.readQuery(conn_DBC, query, dealer_id)
    if no_infos_msg[0][0] == 1 or no_infos_msg[0][0] == "1":
        return no_infos_msg[0][1]
    else:
        return None  

get opening Hours

In [21]:
opening_hours_url = (
        f"https://websites.addstaging.com/api/external_api/hours?user_id={dealer_id}")
headers=create_token()
r = requests.get(opening_hours_url, headers=headers)

get Carfax link from Api

In [22]:
info_veh_url = f"https://inventory.addstaging.com/api/get-carfax-link?user_id={dealer_id}&vin={VIN}"
headers=create_token()
r = requests.get(info_veh_url,headers=headers)

NameError: name 'VIN' is not defined

offer_test_drive

In [23]:
def test_drive_chat_settings(DB, dealer_id):
    try:
        conn_DBC = DB.connexion(
            PWA_DB_HOST_V12CHAT_READ,
            PWA_DB_USERNAME_V12CHAT_READ,
            PWA_DB_PASSWORD_V12CHAT_READ,
            PWA_DB_DATABASE_V12CHAT_READ,
        )
        initiedQuery = """SELECT `offer_test_drive` FROM `chat_settings` WHERE `user_id` = %s"""
        data=dealer_id
        initiedRes = DB.readQuery(conn_DBC, initiedQuery, data)
        if initiedRes[0][0]==0:
            return False
        return True
    except:
        return True

In [24]:
import mysql.connector
import requests
import json

# Assuming these functions and necessary imports are defined in a separate file

# Database connection details
DB_CONFIG = {
    "host": "172.17.0.2",
    "user": "root",
    "password": "admin",
    "database": "bot",
    "port": 3306
}

def save_dealer_info(dealer_id, DB):
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()

    try:
        # Execute all the functions
        shipping_result = _shipping(DB, dealer_id)
        dealer_name_result = dealership_name(DB, dealer_id)
        time_zone = dealer_time_zone(DB, dealer_id)
        address_result = address_location(DB, dealer_id)
        phone = get_dealer_phone(DB, dealer_id)
        inventory_link = inventory_url(DB, dealer_id)
        financing_result = financing_url(DB, dealer_id)
        welcome_message = get_welcome_message(DB, dealer_id)
        no_info_message = message_no_informations(DB, dealer_id)
        offers_test_drive = test_drive_chat_settings(DB, dealer_id)

        # Get opening hours
        opening_hours_url = f"https://websites.addstaging.com/api/external_api/hours?user_id={dealer_id}"
        headers = create_token()
        r = requests.get(opening_hours_url, headers=headers)
        opening_hours = r.json() if r.status_code == 200 else None

        # Truncate or modify opening_hours to fit in the column
        if opening_hours:
            opening_hours_str = json.dumps(opening_hours)
            if len(opening_hours_str) > 255:  # Assuming VARCHAR(255), adjust if different
                opening_hours_str = opening_hours_str[:252] + '...'
        else:
            opening_hours_str = None

        # Prepare data for insertion/update
        data = (
            dealer_name_result,
            address_result[0] if address_result else None,
            phone,
            financing_result[0] if financing_result else None,
            inventory_link,
            offers_test_drive,
            welcome_message,
            shipping_result,
            True,  # trade_ins (always set to True)
            opening_hours_str,
            financing_result[0] is not None if financing_result else False,
            financing_result[1] if financing_result else None,
            "strict",  # bot_behavior (always set to "strict")
            dealer_id
        )

        # Update the data
        update_query = """
        UPDATE dealers_info 
        SET dealer_name = %s,
            address = %s,
            phone = %s,
            credit_app_link = %s,
            inventory_link = %s,
            offers_test_drive = %s,
            welcome_message = %s,
            shipping = %s,
            trade_ins = %s,
            opening_hours = %s,
            offer_finance = %s,
            financing_type = %s,
            bot_behavior = %s
        WHERE dealer_id = %s
        """

        cursor.execute(update_query, data)

        if cursor.rowcount == 0:
            # If no rows were updated, insert a new record
            insert_query = """
            INSERT INTO dealers_info 
            (dealer_name, address, phone, credit_app_link, inventory_link, 
            offers_test_drive, welcome_message, shipping, trade_ins, opening_hours, 
            offer_finance, financing_type, bot_behavior, dealer_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(insert_query, data)

        conn.commit()
        print(f"Information for dealer {dealer_id} has been saved/updated in the database.")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        conn.rollback()

    finally:
        cursor.close()
        conn.close()

# Usage
def main(dealer_id, DB):
    save_dealer_info(dealer_id, DB)

# Call the main function with the required parameters
# main(your_dealer_id, your_DB_object)

In [27]:
db = DataBase()
dealer_id=40255

In [28]:
save_dealer_info(dealer_id, db)

Information for dealer 40255 has been saved/updated in the database.


get last message in conversation of a lead

In [51]:
##function to get last message in conversation of a lead
def last_messages(DB, lead_id):
    conv_id_query = "SELECT id FROM `conversations` WHERE lead_id = %s"
    messages_query = "SELECT content, is_bot, is_bdc FROM `messages` WHERE conversation_id = %s ORDER BY id DESC LIMIT 7; "
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    conv_res = DB.readQuery(conn_DBC, conv_id_query, lead_id)
    con_id = conv_res[0][0]
    msg_res = DB.readQuery(conn_DBC, messages_query, con_id)
    messages = msg_res[::-1]
    last_mesg = ""
    for msg in messages:
        if msg[1] is not None or msg[2] is not None:
            last_mesg += "Bot:" + str(msg[0]) + "\n"
        else:
            last_mesg += "client:" + str(msg[0]) + "\n"
    return last_mesg


In [56]:
print(last_messages(db,10551))

Bot:START CHAT
Bot:Hello, thank you for contacting Car Nation. We’ve got your inquiry for 2012 VOLKSWAGEN GTI.   It is still available.  https://www.carnation-llc.com/inventory/view/17841608/2012-VOLKSWAGEN-GTI- Do you have any questions we may assist you with?

